# Snowflake Connection Demo

This notebook demonstrates how to connect to Snowflake using the `camber.snowflake.connect` function.

This function is designed to securely read credentials from your Camber environment variables, making it easy to connect to your data warehouse without hard-coding sensitive information.

**Prerequisites:**

Before you begin, ensure you have set your Snowflake credentials in https://app.cambercloud.com/data-connectors

In [1]:
# Import necessary libraries
import pandas as pd
import camber.snowflake as sf
from camber.exceptions import CamberEnvironmentException

## 1. Establish Connection

Now, we'll call the `sf.connect()` function. It will automatically read all the environment variables we just checked and attempt to establish a connection.

In [ ]:
conn = None
try:
    conn = sf.connect()
    print("Connection to Snowflake successful!")
    
except CamberEnvironmentException as e:
    print(f"Connection Failed: {e}")
    print("Please ensure your Snowflake environment variables are set correctly.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Connection to Snowflake successful!


## 2. Run a Simple Query

If the connection was successful, we can run a simple query to verify that everything is working.

In [3]:
if conn:
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT CURRENT_VERSION()")
        
        # Fetch the result
        version = cursor.fetchone()[0]
        print(f"Snowflake Version: {version}")
        
    except Exception as e:
        print(f"Query failed: {e}")
    finally:
        # Always close the cursor
        if 'cursor' in locals():
            cursor.close()
else:
    print("Skipping query: Connection object 'conn' was not created.")

Snowflake Version: 9.34.0


## 4. Fetch Data with Pandas

The connection object is compatible with Pandas' `read_sql` function. Let's try to list the databases in your account.

In [4]:
if conn:
    try:
        df = pd.read_sql("SHOW DATABASES LIMIT 10;", conn)
        print("Successfully fetched databases:")
        display(df)
    except Exception as e:
        print(f"Pandas read_sql failed: {e}")
else:
    print("Skipping query: Connection object 'conn' was not created.")

/tmp/ipykernel_480/1565414139.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SHOW DATABASES LIMIT 10;", conn)


Successfully fetched databases:


,created_on,name,is_default,is_current,origin,owner,comment,options,retention_time,kind,owner_role_type,object_visibility
0,2025-10-08 04:24:03.837000+00:00,SNOWFLAKE,N,N,SNOWFLAKE.ACCOUNT_USAGE,,,,0,APPLICATION,,None
1,2025-10-08 04:24:08.325000+00:00,SNOWFLAKE_LEARNING_DB,N,N,,ACCOUNTADMIN,Created by Snowflake during account provisioning,,1,STANDARD,ROLE,None
2,2025-10-08 04:24:07.451000+00:00,SNOWFLAKE_SAMPLE_DATA,N,N,SFSALESSHARED.SFC_SAMPLES_AWSUSEAST2.SAMPLE_DATA,ACCOUNTADMIN,Preloaded TPCH Data provided by Snowflake duri...,,0,IMPORTED DATABASE,ROLE,None
3,2025-10-08 04:31:36.990000+00:00,USER$CAMBERCLOUD,N,N,,,,,1,PERSONAL DATABASE,,None


## 5. Close the Connection

Finally, it's important to close the connection when you're done to free up resources.

In [5]:
if conn:
    try:
        conn.close()
        print("Snowflake connection closed.")
    except Exception as e:
        print(f"Error closing connection: {e}")
else:
    print("No connection to close.")

Snowflake connection closed.
